In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
path = '../PostBornEma/'

In [ ]:
data = np.loadtxt(path+'FFT_k-1Pk.dat',skiprows=2)

#----------------------#
n         = data[:,0]
Re_c_n    = data[:,1]
Im_c_n    = data[:,2]
Re_nu_n   = data[:,3]
Im_nu_n   = data[:,4]
#----------------------#

c_n  = Re_c_n  + 1j * Im_c_n
nu_n = Re_nu_n + 1j * Im_nu_n

In [ ]:
# for cl psipsi
data =  np.load(path+'Il_nu_t_new1-ell-nn-tt.npy')
ell       = data[:,0]
nn        = data[:,1]
tt        = data[:,2]
#----------------------#

data =  np.load(path+'Il_nu_t_new1-ReI-ImI.npy')
Re_I      = data[:,0]
Im_I      = data[:,1]
#----------------------#

I_0 = Re_I + 1j * Im_I


In [ ]:
#for cl psi delta
data =  np.load(path+'Il_nu_t_nd2-ReI-ImI.npy')
Re_I      = data[:,0]
Im_I      = data[:,1]
#----------------------#

I_2 = Re_I + 1j * Im_I


# for cl delta delta
data =  np.load(path+'Il_nu_t_nd4-ReI-ImI.npy')
Re_I      = data[:,0]
Im_I      = data[:,1]
#----------------------#

I_4 = Re_I + 1j * Im_I



In [ ]:
del Re_I, Im_I, data

In [ ]:
#shrink
ell_ = np.unique(ell)
nu_n_= np.unique(nu_n) 
t_   = np.unique(tt)


#make 3D arrays 
ell = ell.reshape(ell_.size, nu_n_.size, t_.size)
nn  = nn.reshape(ell_.size, nu_n_.size, t_.size)
tt  = tt.reshape(ell_.size, nu_n_.size, t_.size)
I_0 = I_0.reshape(ell_.size, nu_n_.size, t_.size)
I_2 = I_2.reshape(ell_.size, nu_n_.size, t_.size)
I_4 = I_4.reshape(ell_.size, nu_n_.size, t_.size)

cn2 = c_n.copy()
cn2[:-1] *=2

#insert a new axis for r and multiply with c_n 
I0_lcrt = np.expand_dims(I_0, 2)*cn2.reshape(1, -1, 1, 1)
I2_lcrt = np.expand_dims(I_2, 2)*cn2.reshape(1, -1, 1, 1)
I4_lcrt = np.expand_dims(I_4, 2)*cn2.reshape(1, -1, 1, 1)

#change t and c
I0_ltrc = np.swapaxes(I0_lcrt, 1, 3)
I2_ltrc = np.swapaxes(I2_lcrt, 1, 3)
I4_ltrc = np.swapaxes(I4_lcrt, 1, 3)


In [ ]:
#get Gaussian quadrature weights
data = np.loadtxt(path+'GG_weights.dat',skiprows=2)
t1   = data[:,0]
w1   = data[:,1]
assert(np.allclose(t_,t1))


In [ ]:
#cosmology
h         = 0.6770
omega_cdm = 0.11923
omega_b   = 0.02247
Omega_b   = omega_b/h**2
Omega_cdm = omega_cdm/h**2
Omega_m   = Omega_b+Omega_cdm
ns        = 0.96824
A0        = 2.10732*10**(-9)

#dictionary for class
cosmo_dict={'h': h,
'omega_b' : omega_b,
'omega_cdm': omega_cdm,
'A_s'    : A0,
'n_s'    : ns,
'k_pivot' : 0.05}
print(cosmo_dict)

# speed of light
c      = 299792458./1000. # km/s

#prefactor for Cl_kk computation from Cl_dd
prefac = 1.5*Omega_m*(100.)**2/c**2 #without h

In [ ]:
#interpolation splines for redshift and distance
from classy import Class
cosmo = Class()
cosmo.set(cosmo_dict)
cosmo.compute()
cosmo_b               = cosmo.get_background()

class_z               = cosmo_b['z'][::-1]
class_chi             = cosmo_b['comov. dist.'][::-1]
class_D               = cosmo_b['gr.fac. D'][::-1]
class_H               = cosmo_b['H [1/Mpc]'][::-1]/h #already divided by c
derivParams           = cosmo.get_current_derived_parameters(['z_rec'])

dchi_dz=(class_chi[1::]-class_chi[0:-1])/(class_z[1::]-class_z[0:-1])*h
z_mean = (class_z[1::]+class_z[0:-1])/2

z_cmb                 = derivParams['z_rec']


from scipy.interpolate import interp1d
chi_z = interp1d(class_z,class_chi*h,fill_value=0, bounds_error=False)
z_chi = interp1d(class_chi*h,class_z,fill_value=0, bounds_error=False)  # Mpc/h
D_chi = interp1d(class_chi*h,class_D,fill_value=0, bounds_error=False)    # growth
D_z   = interp1d(class_z,class_D,fill_value=0, bounds_error=False)
# chi_cmb
chi_cmb = chi_z(z_cmb)

dz_dchi = interp1d(class_chi*h,class_H,fill_value=0, bounds_error=False)
dchi_dz = interp1d(z_mean,dchi_dz,fill_value=0, bounds_error=False)


In [ ]:
# sanity checks
z = np.linspace(0,5,100)
chi = chi_z(z)
plt.figure()
plt.plot(class_z,class_D,ls='-')
plt.plot(z,D_z(z),ls='--')
plt.plot(z,1/(1+z),ls='-.')
plt.xlim(0,5)
plt.show()
D_z(0)
z=np.linspace(0.,10,200)
chi = chi_z(z)
plt.figure()
plt.loglog(z,chi_z(z))
plt.show()

plt.figure()
plt.plot(z,D_z(z)/D_chi(chi_z(z))-1.)
plt.show()
plt.figure()
plt.loglog(z,)
plt.show()
plt.figure()
plt.loglog(z,z_chi(chi_z(z)))
plt.show()
print(z.shape,chi.shape)
plt.figure()
plt.loglog(chi,dchi_dz(z)**(-1))
plt.loglog(chi,dz_dchi(chi))
plt.loglog(class_chi*h,class_H,ls='--')
plt.ylim(1e-4,1e-1)
plt.show()

# Integrals

In [ ]:
from scipy.integrate import simps, quadrature
from scipy.interpolate import interp1d
import time


In [ ]:
# CL^kk following Eq. 3.5 in https://arxiv.org/abs/1705.05022

def lensing_kernel(xi, xmax):
    return (xmax - xi)/(xmax*xi) * (xmax > xi) * (1.+z_chi(xi))


r2d, t2d = np.meshgrid(t_,t_)
w11, w12 = np.meshgrid(w1,w1)
# inflate by one dimensions (nu_n)
r2d, t2d = np.expand_dims(r2d, 2), np.expand_dims(t2d, 2)
w11, w12 = np.expand_dims(w11, 2), np.expand_dims(w12, 2)
#I0_ltrc  = np.swapaxes(I0_lcrt, 1, 3)


begin=time.time()

chi1fac0 = (lensing_kernel(r2d*chi_cmb, chi_cmb)*D_chi(r2d*chi_cmb))
chi1fac0 = chi1fac0 *(r2d*chi_cmb)**(1-nu_n_.reshape(1, 1, -1))

chi2fac00 = (lensing_kernel(t2d*r2d*chi_cmb, chi_cmb)*D_chi(r2d*t2d*chi_cmb))
chi2fac01 = (lensing_kernel(1./t2d*r2d*chi_cmb, chi_cmb)*D_chi(r2d*1./t2d*chi_cmb))
chi2fac01 = chi2fac01 * t2d**(nu_n_.reshape(1, 1, -1)-2)
chi2fac0  = chi2fac00 + chi2fac01

chifacs = w11*w12*chi1fac0* chi2fac0

result=np.zeros_like(ell_)
lmax = ell_.size
for ii  in range(ell_.size):        
    result[ii] = np.sum(chifacs*I0_ltrc[ii])

Cl=chi_cmb * result*1./np.pi**2*prefac**2/2.

print('Time taken = ', time.time()-begin)

We have integrals of the form (for example, take the second term of $C_{22shift}^{\kappa g}$ i.e. eq. 4.2; drop the integral over $\ell$):
<br>
\begin{eqnarray}
Target &=& \int_0^{\chi_c}  d\chi W(\chi, \chi_c)\int_0^{\chi_0} d\chi' W_h(\chi') C_{l1}^{\Phi \Psi}(\chi', \chi) C_{l2}^{\delta\phi}(\chi', \chi) \\
&=&\int_0^{\chi_c} d\chi W(\chi, \chi_c)  \int_0^{\chi_0} d\chi' W_h(\chi')
\int_0^{\chi'} d\chi" W(\chi", \chi') C_{l1}^{\Psi \Psi}(\chi", \chi) 
\int_0^{\chi} d\chi''' W(\chi''', \chi) C_{l2}^{\delta \Psi}(\chi', \chi''') 
\end{eqnarray}

Now,
\begin{eqnarray}
C_l^{ab}(\chi_1, \chi_2) &=& \int \frac{dk}{k} j_l(k\chi_1)j_l(k\chi_2) [k^3 P^{ab}(k, \chi_1, \chi_2)] \\
&=& \sum_n c_n \chi_1 ^{-\nu_n} I_l(\nu_n, \frac{\chi_2}{\chi_1})
\end{eqnarray}

So, 
\begin{eqnarray}
Target &=& \int_0^{\chi_c} d\chi W(\chi, \chi_c)  \int_0^{\chi_0} d\chi' W_h(\chi') \\
&\times& \Big(\int_0^{\chi'} d\chi" W(\chi", \chi') \sum_n c^{ab}_n \chi ^{-\nu_n} I_{l1}^{ab}(\nu_n, \frac{\chi''}{\chi}) \Big) \\
&\times& \Big(\int_0^{\chi} d\chi''' W(\chi''', \chi) \sum_n c^{cd}_n \chi' ^{-\nu_n} I_{l2}^{cd}(\nu_n, \frac{\chi'''}{\chi'}) \Big)
\end{eqnarray}

These are coupled together in complicated way amongst the 4 $\chi$. 
<br>
I think the best way to estimate these are to save the array - 
$$
\int_0^{\chi} d\chi" W(\chi", \chi) \sum_n c_n \chi' ^{-\nu_n} I_l(\nu_n, \frac{\chi''}{\chi'}) 
$$
in the above above equation as a matrix $G(l, \chi, \chi')$.
<br>
This then reduces the evaluation of integrals to the sum - 
\begin{eqnarray}
Target &=& \int_0^{\chi_c} d\chi W(\chi, \chi_c)  \int_0^{\chi_0} d\chi' W_h(\chi') \times G^{ab}(l_1, \chi', \chi)\times G^{cd}(l_2, \chi, \chi')\\
\end{eqnarray}
which we can evaluate as a Gauss Qaud sum.

To save $G^{cd}(l_2, \chi, \chi')$, we have -

\begin{eqnarray}
G^{cd}(l, \chi, \chi') &=& \int_0^{\chi} d\chi" W(\chi", \chi) \sum_n c_n \chi' ^{-\nu_n} I_l(\nu_n, \frac{\chi''}{\chi'}) \\
t &=& \chi ''/\chi' \\
G^{cd}(l, \chi, \chi') &=& \int_0^{\chi/\chi'} dt W(t\chi', \chi) \sum_n c_n \chi' ^{1-\nu_n} I_l(\nu_n, t) \\
G^{cd}(l, \chi, \chi') &=& \sum_n  \int_0^1 dt \big( c_n(\chi', t\chi') (W(t\chi', \chi) + t^{-\nu-2} c_n(\chi', \chi'/t) W(\chi'/t, \chi) \big) \chi' ^{1-\nu_n} I_l(\nu_n, t) \\
\end{eqnarray}

__The problem here is that $t$ is fixed. And the obvious values of $\chi$ and $\chi'$ that we should take are $t \chi_{CMB}$. Hence I am a bit uncertain how well are we sampling this integral anymore in going from $\chi/\chi'$ to (0-1) limit. But it should work since that is what we do in the Cl_kk as well
__

In [ ]:
### Estimate as function of 'chi' and then integrate over \chi
###This is a test to check that we can estimate Cl_kk with this accurately, which kind of justifies that we are estimating
###The G matrix accurately.

# def Wkernel(xi, xmax):
#     return (xmax - xi)/(xmax*xi) * (xmax > xi) * (1.+z_chi(xi))


chimax = chi_cmb
chi1s  = t_*chimax

chi1fac0 = (lensing_kernel(chi1s, chimax))#*D_chi(chi1s)) #Move this inside the sum
#ltc
t2d    = t_.reshape(1, -1, 1)
#get rid of r -axis
I0_ltc = np.squeeze(I0_ltrc)

begin = time.time()
result = []
for ii, chi in enumerate(chi1s):
    
    chifac    = chi**(1-nu_n_.reshape(1, 1, -1)) * D_chi(chi) #Move this c_n inside the loop
    
    chi2fac00 = (lensing_kernel(t2d*chi, chimax)*D_chi(t2d*chi))
    chi2fac01 = (lensing_kernel(1./t2d*chi, chimax)*D_chi(1./t2d*chi))* t2d**(nu_n_.reshape(1,  -1)-2)
    chi2fac0  = chi2fac00 + chi2fac01
    fac       = w1.reshape(1, -1, 1)*chi2fac0*chifac
    integrand = fac * I0_ltc
    result.append(integrand.sum(axis = (1, 2)))

result = np.array(result)
print(result.shape)
result = (result*chi1fac0.reshape(-1, 1)*w1.reshape(-1, 1)).sum(axis=0)
Cl2=chimax * result*4./np.pi**2*prefac**2/8.

print('Time taken = ', time.time()-begin)

In [ ]:
mean = 3
width = 0.5
cosmo_dict['output']='nCl, lCl, tCl, sCl, mPk'
cosmo_dict['lensing']='yes'
cosmo_dict['selection']='gaussian'
cosmo_dict['selection']='gaussian'
cosmo_dict['selection_mean'] = str(mean)
cosmo_dict['selection_width'] = str(width)
cosmo_dict['l_switch_limber']=100
cosmo_dict['perturb_sampling_stepsize']=0.01
cosmo_dict['P_k_max_1/Mpc']=10.

cosmo.empty()
cosmo.set(cosmo_dict)
cosmo.compute()

In [ ]:
cldd = cosmo.density_cl(300)
cllen = cosmo.lensed_cl(2000)



In [ ]:
ll,clpp,clpp2 = np.loadtxt('class_ouput.out')
plt.figure()
plt.loglog(ell_[:lmax],(ell_[:lmax]*(ell_[:lmax]+1.))**2*Cl[:lmax]/(2.*np.pi))
plt.loglog(ell_[:lmax],(ell_[:lmax]*(ell_[:lmax]+1.))**2*Cl2[:lmax]/(2.*np.pi))
plt.loglog(ll,(ll*(ll+1.))**2*clpp/(2.*np.pi),ls='--')
plt.loglog(ll,(ll*(ll+1.))**2*np.interp(ll,cllen['ell'],cllen['pp'])/(2.*np.pi),ls=':')
# plt.loglog(ell_[:lmax],(ell_[:lmax]*(ell_[:lmax]+1.))**2*Cl3[:lmax]/(2.*np.pi))
plt.xlim(2,500)

In [ ]:
ll,clpp,clpp2 = np.loadtxt('class_ouput.out')

plt.figure()
plt.semilogx(ell_,Cl/np.interp(ell_,ll,clpp), ls=":", lw=3, label='mcSpeedy')
plt.semilogx(ell_,Cl2/np.interp(ell_,ll,clpp), ls="--", lw=2, label='chi')
plt.semilogx(ell_,Cl2/np.interp(ell_,cllen['ell'],cllen['pp']), ls="--", lw=2, label='chi')
plt.ylim(0.95,1.05)
# plt.ylim(0.9,2)
plt.grid()
plt.legend()

# save xifunc as described above

To save $G^{cd}(l_2, \chi, \chi')$, we have -

\begin{eqnarray}
G^{cd}(l, \chi, \chi') &=& \int_0^{\chi} d\chi" W(\chi", \chi) \sum_n c_n \chi' ^{-\nu_n} I_l(\nu_n, \frac{\chi''}{\chi'}) \\
t &=& \chi ''/\chi' \\
G^{cd}(l, \chi, \chi') &=& \int_0^{\chi/\chi'} dt W(t\chi', \chi) \sum_n c_n \chi' ^{1-\nu_n} I_l(\nu_n, t) \\
G^{cd}(l, \chi, \chi') &=& \sum_n  \int_0^1 dt \big( c_n(\chi', t\chi') (W(t\chi', \chi) + t^{-\nu-2} c_n(\chi', \chi'/t) W(\chi'/t, \chi) \big) \chi' ^{1-\nu_n} I_l(\nu_n, t) \\
\end{eqnarray}

__The problem here is that $t$ is fixed. And the obvious values of $\chi$ and $\chi'$ that we should take are $t \chi_{CMB}$. Hence I am a bit uncertain how well are we sampling this integral anymore in going from $\chi/\chi'$ to (0-1) limit. But it should work since that is what we do in the Cl_kk as well
__

In [ ]:
# ### Saving here

# def Wkernel(xi, xmax):
#     return (xmax - xi)/(xmax*xi) * (xmax > xi) * (1.+z_chi(xi))

# I4_ltc = np.squeeze(I4_ltrc)
# t2d = t_.reshape(1, -1, 1)

# chis = t_*chi_cmb
# chips = t_*chi_cmb

# Gsave = []

# begin = time.time()
# for ichim, chimax in enumerate(chis[:]):
#     if ichim %10 == 0:
#         print(ichim)
        
#     chi1fac0 = (Wkernel(chips, chimax))#*D_chi(chi1s)) #Move this inside the sum

#     result = []
#     for ii, chi in enumerate(chips):

#         chifac = chi**(1-nu_n_.reshape(1, 1, -1)) * D_chi(chi) #Move this c_n inside the loop

#         chi2fac00 = (Wkernel(t2d*chi, chimax)*D_chi(t2d*chi))
#         chi2fac01 = (Wkernel(1/t2d*chi, chimax)*D_chi(1/t2d*chi))* t2d**(nu_n_.reshape(1,  -1)-2)
#         chi2fac0 = chi2fac00 + chi2fac01
#         fac = w1.reshape(1, -1, 1)*chi2fac0*chifac
#         integrand = fac * I4_ltc
#         result.append(integrand.sum(axis = (1, 2)))

#     Gsave.append(np.array(result).T)
#     if ichim %10 == 0:
#         print(time.time()-begin)
    
# Gsave = np.swapaxes(np.array(Gsave), 0, 1)

# np.save(path + 'G-Ilnut_nd4', Gsave)

# Clgg

In [ ]:
# Chirag's kernel definition
def old_kernel(z0, sigz=0.3):
    z1, z2   = z0-sigz/2., z0+sigz/2.
    xi1, xi2 = chi_z(z1), chi_z(z2)
    sigxi    = xi2-xi1
    xi0     = chi_z(z0)
    def kernel(xi):
        return 1./np.sqrt(2*np.pi*sigxi**2) * np.exp(- (xi-xi0)**2/2/sigxi**2) 
    return kernel

# general kernel if dNdz is Gaussian
def Gauss_redshift(z0,sigma_z):
    def z_kernel(z):
        return 1./np.sqrt(2.*np.pi)/sigma_z*np.exp(-(z-z0)**2/2./sigma_z**2)
    return z_kernel

def gal_kernel(z_kernel,ximax=5):
    def chi_kernel(xi):
        return z_kernel(z_chi(xi))*dz_dchi(xi)
    return chi_kernel

# kernel as in paper?
def Gauss_chi(z0,sigma_z,chimax=5):
    chi0      = chi_z(z0)
    sigma_chi = sigma_z/dz_dchi(chi0)
    def chi_kernel(chi):
        return 1./np.sqrt(2.*np.pi)/sigma_chi*np.exp(-(chi-chi0)**2/2./sigma_chi**2)
    return chi_kernel
    

# supports redshift dependent bias, for now: constant
def bias_chi(chi):
    return 1.9

    
#First kernel
###############################################################lcr##########
#Set up

r2d, t2d = np.meshgrid(t_,t_)
w11, w12 = np.meshgrid(w1,w1)
#nu axis
r2d, t2d = np.expand_dims(r2d, 2), np.expand_dims(t2d, 2)
w11, w12 = np.expand_dims(w11, 2), np.expand_dims(w12, 2)
I4_ltrc  = np.swapaxes(I4_lcrt, 1, 3)


#Redshift kernels

# uncomment to reproduce left upper plot
#z10, z20, sigz = 1., 1., 0.05

z10, z20, sigz = 1., 1.25, 0.5

kernel1 = old_kernel(z10,sigz)
kernel2 = old_kernel(z20,sigz)

#first kernel
#########


begin=time.time()
#Try different chimax to see how sampling changes things
chimax   = chi_z(z20+5.*sigz)

chi1fac0 = (kernel1(r2d*chimax)*D_chi(r2d*chimax)*bias_chi(r2d*chimax))
chi1fac0 = chi1fac0 *(r2d*chimax)**(1-(4+nu_n_.reshape(1, 1, -1)))

chi2fac00 = (kernel2(t2d*r2d*chimax)*D_chi(r2d*t2d*chimax)*bias_chi(r2d*t2d*chimax))
chi2fac01 = (kernel2(1./t2d*r2d*chimax)*D_chi(r2d/t2d*chimax)*bias_chi(r2d/t2d*chimax))
chi2fac01 = chi2fac01 * t2d**((4+nu_n_).reshape(1, 1, -1)-2)
chi2fac0  = chi2fac00 + chi2fac01

chifacs = w11*w12*chi1fac0* chi2fac0

result=np.zeros_like(ell_)
lmax = ell_.size
for ii  in range(ell_.size):        
    result[ii] = np.sum(chifacs*I4_ltrc[ii])
    

Clgg = chimax * result*1./np.pi**2/2.

print('Time taken = ', time.time()-begin)


#Second kernel
###################################
begin=time.time()
kernel1 = Gauss_chi(z10,sigz)
kernel2 = Gauss_chi(z20,sigz)

chimax  = chi_cmb

chi1fac0 = (kernel1(r2d*chimax)*D_chi(r2d*chimax)*bias_chi(r2d*chimax))
chi1fac0 = chi1fac0 *(r2d*chimax)**(1-(4+nu_n_.reshape(1, 1, -1)))

chi2fac00 = (kernel2(t2d*r2d*chimax)*D_chi(r2d*t2d*chimax)*bias_chi(r2d*t2d*chimax))
chi2fac01 = (kernel2(1/t2d*r2d*chimax)*D_chi(r2d*1/t2d*chimax)*bias_chi(r2d/t2d*chimax))
chi2fac01 = chi2fac01 * t2d**((4+nu_n_).reshape(1, 1, -1)-2)
chi2fac0  = chi2fac00 + chi2fac01


chifacs = w11*w12*chi1fac0* chi2fac0

result=np.zeros_like(ell_)
lmax = ell_.size
for ii  in range(ell_.size):        
    result[ii] = np.sum(chifacs*I4_ltrc[ii])

Clggcmb = chimax * result/np.pi**2/2.

print('Time taken = ', time.time()-begin)

#third kernel
###################################
begin=time.time()
kernel1 = gal_kernel(Gauss_redshift(z10,sigz))
kernel2 = gal_kernel(Gauss_redshift(z20,sigz))

chimax = chi_cmb

chi1fac0 = (kernel1(r2d*chimax)*D_chi(r2d*chimax)*bias_chi(r2d*chimax))
chi1fac0 = chi1fac0 *(r2d*chimax)**(1-(4+nu_n_.reshape(1, 1, -1)))

chi2fac00 = (kernel2(t2d*r2d*chimax)*D_chi(r2d*t2d*chimax)*bias_chi(r2d*t2d*chimax))
chi2fac01 = (kernel2(1/t2d*r2d*chimax)*D_chi(r2d*1/t2d*chimax)*bias_chi(r2d/t2d*chimax))
chi2fac01 = chi2fac01 * t2d**((4+nu_n_).reshape(1, 1, -1)-2)
chi2fac0  = chi2fac00 + chi2fac01


chifacs = w11*w12*chi1fac0* chi2fac0

result=np.zeros_like(ell_)
lmax = ell_.size
for ii  in range(ell_.size):        
    result[ii] = np.sum(chifacs*I4_ltrc[ii])

Clggcmb_2 = chimax * result/np.pi**2/2.

print('Time taken = ', time.time()-begin)


In [ ]:
# uncomment for plotting left upper plot

# plot for 3 different kernel definitions (chirag's, paper and mine)
# result is very sensitive to kernel, but also off by a factor of a few
# plt.plot(ell_, abs(Clgg)*1e6/4./np.pi)
# plt.plot(ell_, abs(Clggcmb)*1e6/4./np.pi,ls='--')
# plt.plot(ell_, abs(Clggcmb_2)*1e6/4./np.pi,ls='-.')
# plt.loglog()
# plt.ylim(0.3, 12)
# # plt.yticks((0.5, 1, 2, 5, 10), (0.5, 1, 2, 5, 10))
# #plt.yticks(np.logspace(-2, 2, 5), np.logspace(-2, 2, 5))
# plt.xlim(2, 500)
# plt.grid(which='both')

# plotting upper righ plot in Fig 2. Differnce seems a factor of 1/4/pi
plt.plot(ell_, abs(Clgg)*1e7/4./np.pi)
plt.plot(ell_, abs(Clggcmb)*1e7/4./np.pi,ls='--')
plt.plot(ell_, abs(Clggcmb_2)*1e7/4./np.pi,ls='-.')
plt.loglog()
plt.ylim(0.01, 10)
# plt.yticks((0.5, 1, 2, 5, 10), (0.5, 1, 2, 5, 10))
#plt.yticks(np.logspace(-2, 2, 5), np.logspace(-2, 2, 5))
plt.xlim(2, 500)
plt.grid(which='both')

In [ ]:
plt.plot(ell_, Clggcmb/Clgg)
plt.xscale('log')
plt.ylim(0.90, 1.1)
plt.grid(which='both')

In [ ]:
# computing clkk from Pk_m, approximation where los derivatives are neglected

def lensing_kernel_2(xi, xmax):
    return (xmax - xi)/(xmax)*xi* (xmax > xi) * (1.+z_chi(xi))

kernel = lensing_kernel_2

r2d, t2d = np.meshgrid(t_,t_)
w11, w12 = np.meshgrid(w1,w1)
#nu axis
r2d, t2d = np.expand_dims(r2d, 2), np.expand_dims(t2d, 2)
w11, w12 = np.expand_dims(w11, 2), np.expand_dims(w12, 2)
I4_ltrc  = np.swapaxes(I4_lcrt, 1, 3)

begin=time.time()
#Try different chimax to see how sampling changes things
chimax   = chi_cmb

chi1fac0 = (kernel(r2d*chimax,chi_cmb)*D_chi(r2d*chimax))
chi1fac0 = chi1fac0 *(r2d*chimax)**(1-(4+nu_n_.reshape(1, 1, -1)))

chi2fac00 = (kernel(t2d*r2d*chimax,chi_cmb)*D_chi(r2d*t2d*chimax))
chi2fac01 = (kernel(1./t2d*r2d*chimax,chi_cmb)*D_chi(r2d/t2d*chimax))
chi2fac01 = chi2fac01 * t2d**((4+nu_n_).reshape(1, 1, -1)-2)
chi2fac0  = chi2fac00 + chi2fac01

chifacs = w11*w12*chi1fac0* chi2fac0

result=np.zeros_like(ell_)
lmax = ell_.size
for ii  in range(ell_.size):        
    result[ii] = np.sum(chifacs*I4_ltrc[ii])
    

Clkk = chimax * result*1./np.pi**2/2.*prefac**2

print('Time taken = ', time.time()-begin)

In [ ]:
# difference to full sky is not as bad as to flat sky (this is also noted in the Kilbinger (Kitching?) paper for the Limber approximation)
plt.figure()
plt.loglog(ell_[:lmax],(ell_[:lmax]*(ell_[:lmax]+1.))**2*Cl[:lmax]/(2.*np.pi),label='from psi full sky')
plt.loglog(ell_[:lmax],(ell_[:lmax]*(ell_[:lmax]+1.))**2*Cl2[:lmax]/(2.*np.pi),label='from psi full sky 2')
plt.loglog(ll,(ll*(ll+1.))**2*clpp/(2.*np.pi),ls='--', label='class old*4')
plt.loglog(ll,(ll*(ll+1.))**2*np.interp(ll,cllen['ell'],cllen['pp'])/(2.*np.pi),ls=':',label='class new *4')
plt.loglog(ell_[:lmax],Clkk[:lmax]/(2.*np.pi),ls='--',label='from delta flat sky')
plt.loglog(ell_[:lmax],(ell_[:lmax]*(ell_[:lmax]))**2*Cl2[:lmax]/(2.*np.pi),label='from psi flat sky')
# plt.loglog(ell_[:lmax],(ell_[:lmax]*(ell_[:lmax]+1.))**2*Cl3[:lmax]/(2.*np.pi))
plt.xlim(2,500)
plt.legend()
plt.show()


# Integrals

\begin{eqnarray}
C_l^{ab}(\chi_1, \chi_2) &=& \int \frac{dk}{k} j_l(k\chi_1)j_l(k\chi_2) [k^3 P^{ab}(k, \chi_1, \chi_2)] \\
&=& \sum_n c_n \chi_1 ^{-\nu_n} I_l(\nu_n, \frac{\chi_2}{\chi_1})
\end{eqnarray}
(dropped 1/$4\pi$)

\begin{eqnarray}
G^{ab}(l, \chi, \chi') &=& \int_0^{\chi} d\chi" W(\chi", \chi) C_l^{ab}(\chi', \chi")  \\
&=&\int_0^{\chi} d\chi" W(\chi", \chi) \sum_n c_n \chi' ^{-\nu_n} I_l^{ab}(\nu_n, \frac{\chi''}{\chi'}) 
\end{eqnarray}
<br>


In the following, drop the integral over $\ell$

- $C_{22shift, A}^{\kappa g}$ i.e. eq. 4.2 second term :
<br>
\begin{eqnarray}
C_{22shift, A}^{\kappa g} &=& \int_0^{\chi_c}  d\chi W(\chi, \chi_c)\int_0^{\chi_0} d\chi' W_h(\chi')
C_{l1}^{\delta \Psi}(\chi', \chi) C_{l2}^{\phi\phi}(\chi', \chi) \\
&=&\int_0^{\chi_c} d\chi W(\chi, \chi_c)  \int_0^{\chi_0} d\chi' W_h(\chi') C_{l1}^{\delta \Psi}(\chi', \chi) 
\int_0^{\chi'} d\chi" W(\chi", \chi') 
\int_0^{\chi} d\chi''' W(\chi''', \chi) C_{l2}^{\Psi \Psi}(\chi', \chi''') 
\end{eqnarray}

Notes - here, we can create a table of ($\chi, \chi'$) for the second half i.e. $\int_0^{\chi'} d\chi" W(\chi", \chi') 
\int_0^{\chi} d\chi''' W(\chi''', \chi) C_{l2}^{\Psi \Psi}(\chi', \chi''') $, then simply multiply with other $I_{l1}^{\delta \Psi}$ and do the remaining 2 integrals with GC
 

- $C_{22shift, B}^{\kappa g}$ i.e. eq. 4.2 second term :
<br>
\begin{eqnarray}
C_{22shift, B}^{\kappa g} &=& \int_0^{\chi_c}  d\chi W(\chi, \chi_c)\int_0^{\chi_0} d\chi' W_h(\chi')
C_{l1}^{\Phi \Psi}(\chi', \chi) C_{l2}^{\delta\phi}(\chi', \chi) \\
&=&\int_0^{\chi_c} d\chi W(\chi, \chi_c)  \int_0^{\chi_0} d\chi' W_h(\chi')
\int_0^{\chi'} d\chi" W(\chi", \chi') C_{l1}^{\Psi \Psi}(\chi", \chi) 
\int_0^{\chi} d\chi''' W(\chi''', \chi) C_{l2}^{\delta \Psi}(\chi', \chi''') 
\end{eqnarray}

Notes - here, we can create use table of $G_{l_2, \chi, \chi'}^{\delta \Psi}$ and $G_{l_1, \chi, \chi'}^{\Psi \Psi}$, multiply them and then do the the remaining 2 integrals with GC

- $C_{13ashift, A}^{\kappa g}$ i.e. eq. 4.3 :
<br>
\begin{eqnarray}
C_{13ashift, A}^{\kappa g} &=& \int_0^{\chi_c}  d\chi W(\chi, \chi_c)\int_0^{\chi_0} d\chi' W_h(\chi')
C_{l1}^{\delta \Psi}(\chi', \chi) C_{l2}^{\phi\phi}(\chi, \chi) \\
&=&\int_0^{\chi_c} d\chi W(\chi, \chi_c)  \int_0^{\chi_0} d\chi' W_h(\chi') C_{l1}^{\delta \Psi}(\chi', \chi) 
\int_0^{\chi} d\chi" W(\chi", \chi) 
\int_0^{\chi} d\chi''' W(\chi''', \chi) C_{l2}^{\Psi \Psi}(\chi", \chi''') 
\end{eqnarray}

Notes - here again, same as 22A, we can create a table of ($\chi$) for the second half i.e. $\int_0^{\chi} d\chi" W(\chi", \chi) 
\int_0^{\chi} d\chi''' W(\chi''', \chi) C_{l2}^{\Psi \Psi}(\chi, \chi''') $, put it in the kernel of $\chi$; integrate out $\chi'$ first with GC and then do the full $\chi$ integral with GC


- $C_{31ashift, B}^{\kappa g}$ i.e. eq. 4.4 :
<br>
\begin{eqnarray}
C_{31ashift, B}^{\kappa g} &=& \int_0^{\chi_c}  d\chi W(\chi, \chi_c) \int_0^{\chi_0} d\chi' W_h(\chi')
C_{l1}^{\Phi \Psi}(\chi, \chi) C_{l2}^{\delta\phi}(\chi', \chi) \\
&=&\int_0^{\chi_c} d\chi W(\chi, \chi_c)  \int_0^{\chi_0} d\chi' W_h(\chi')
\int_0^{\chi} d\chi" W(\chi", \chi) C_{l1}^{\Psi \Psi}(\chi", \chi) 
\int_0^{\chi} d\chi''' W(\chi''', \chi) C_{l2}^{\delta \Psi}(\chi', \chi''') 
\end{eqnarray}

Notes - here again, we can do integral over $\chi'$ and $\chi"$ trivially, combine everything into the kernel of $\chi$ and then estimate that with GC


- $C_{13shift}^{\kappa g}$ i.e. eq. 4.5 :
<br>
\begin{eqnarray}
C_{13shift}^{\kappa g} &=& \int_0^{\chi_c}  d\chi W(\chi, \chi_c) \int_0^{\chi_0} d\chi' W_h(\chi')
C_{l1}^{\delta \Psi}(\chi', \chi) C_{l2}^{\phi\phi}(\chi', \chi') \\
&=&\int_0^{\chi_c} d\chi W(\chi, \chi_c)  \int_0^{\chi_0} d\chi' W_h(\chi') C_{l1}^{\delta \Psi}(\chi', \chi) 
\int_0^{\chi'} d\chi" W(\chi", \chi') 
\int_0^{\chi'} d\chi''' W(\chi''', \chi') C_{l2}^{\Psi \Psi}(\chi", \chi''') 
\end{eqnarray}

Notes - here again, same as 31a,A, except we can create a table of ($\chi'$) for the second half i.e. $\int_0^{\chi'} d\chi" W(\chi", \chi') 
\int_0^{\chi'} d\chi''' W(\chi''', \chi') C_{l2}^{\Psi \Psi}(\chi", \chi''') $, put it in the kernel of $\chi'$; then integrate out $\chi'$ and $\chi$ in the usual manner


- $C_{13shift}^{\kappa g}$ i.e. eq. 4.6 :
<br>
\begin{eqnarray}
C_{13shift}^{\kappa g} &=& \int_0^{\chi_c}  d\chi W(\chi, \chi_c) \int_0^{\chi_0} d\chi' W_h(\chi')
\int_0^{\chi}  d\chi" W(\chi", \chi) C_{l1}^{\Psi \Psi}(\chi", \chi) C_{l2}^{\phi\delta}(\chi", \chi') \\
&=&\int_0^{\chi_c} d\chi W(\chi, \chi_c)  \int_0^{\chi_0} d\chi' W_h(\chi') 
\int_0^{\chi}  d\chi" W(\chi", \chi) C_{l1}^{\Psi \Psi}(\chi", \chi)
\int_0^{\chi"} d\chi''' W(\chi''', \chi") C_{l2}^{\Psi\delta}(\chi''', \chi')  
\end{eqnarray}

Notes - here we can do the $\chi', \chi'''$ integrals first  in the usual fashion to create a table of $\chi"$ and then use them as weights to do $\chi, \chi''$ integrals later

- $C_{31bshift}^{\kappa g}$ i.e. eq. 4.6-3.7 :
<br>
\begin{eqnarray}
C_{31bshift}^{\kappa g} &=& \int_0^{\chi_c}  d\chi W(\chi, \chi_c) \int_0^{\chi_0} d\chi' W_h(\chi')
\int_0^{\chi}  d\chi" W(\chi", \chi) C_{l1}^{\Psi \Psi}(\chi", \chi) C_{l2}^{\phi\delta}(\chi", \chi') \\
&=&\int_0^{\chi_c} d\chi W(\chi, \chi_c)  \int_0^{\chi_0} d\chi' W_h(\chi') 
\int_0^{\chi}  d\chi" W(\chi", \chi) C_{l1}^{\Psi \Psi}(\chi", \chi)
\int_0^{\chi"} d\chi''' W(\chi''', \chi") C_{l2}^{\Psi\delta}(\chi''', \chi')  
\end{eqnarray}

Notes - here we can do the $\chi', \chi'''$ integrals first  in the usual fashion to create a table of $\chi"$ and then use them as weights to do $\chi, \chi''$ integrals later

- $C_{13bshift}^{\kappa g}$ i.e. eq. 4.8 :
<br>
\begin{eqnarray}
C_{13shift}^{\kappa g} &=& \int_0^{\chi_c}  d\chi W(\chi, \chi_c) \int_0^{\chi_0} d\chi' W_h(\chi')
\int_0^{\chi'}  d\chi" W(\chi", \chi) C_{l1}^{\Psi \Psi}(\chi", \chi) ...
\end{eqnarray}

Notes - ...

- $C_{13, J}^{\kappa g}$ i.e. eq. 4.8 :
<br>
\begin{eqnarray}
C_{13shift}^{\kappa g} &=& \int_0^{\chi_c}  d\chi W(\chi, \chi_c) \int_0^{\chi_0} d\chi' W_h(\chi')
\int_0^{\chi'}  d\chi" W(\chi", \chi) C_{l1}^{\Psi \Psi}(\chi", \chi) C_{l2}^{\phi\delta} ...
\end{eqnarray}

Notes - ...

In [ ]:

plt.figure()
#full sky
plt.loglog(ll,(ll*(ll+1.))**2*clpp/(2.*np.pi),ls='--',label='class old')
plt.loglog(ell_[:lmax],(ell_[:lmax]*(ell_[:lmax]+1.))**2*Cl[:lmax]/(2.*np.pi),label='from psi full sky')
plt.loglog(ell_[:lmax],(ell_[:lmax]*(ell_[:lmax]+1.))**2*Cl2[:lmax]/(2.*np.pi),label='from psi full sky 2', ls ='--')
#flat sky
plt.loglog(ell_[:lmax],(ell_[:lmax]*(ell_[:lmax]))**2*Cl2[:lmax]/(2.*np.pi),label='from psi flat sky')
plt.loglog(ell_[:lmax],Clkk[:lmax]/(2.*np.pi),ls='--',label='from delta flat sky')
plt.legend()
plt.xlim(2,500)
plt.show()

In [ ]:
plt.loglog(cllen['ell']*cllen['ell'],(cllen['ell']*(cllen['ell']+1))**2*cllen['pp'],ls=':',label='clphiphi from CLASS')
plt.legend()

In [ ]:
# general kernel if dNdz is Gaussian
def Gauss_redshift(z0,sigma_z):
    def z_kernel(z):
        return 1./np.sqrt(2.*np.pi)/sigma_z*np.exp(-(z-z0)**2/2./sigma_z**2)
    return z_kernel

def gal_kernel(z_kernel,ximax=5):
    def chi_kernel(xi):
        return z_kernel(z_chi(xi))*dz_dchi(xi)
    return chi_kernel

#third kernel
###################################
begin=time.time()
kernel1 = gal_kernel(Gauss_redshift(mean,width))
kernel2 = gal_kernel(Gauss_redshift(mean,width))

chimax = chi_cmb

chi1fac0 = (kernel1(r2d*chimax)*D_chi(r2d*chimax))
chi1fac0 = chi1fac0 *(r2d*chimax)**(1-(4+nu_n_.reshape(1, 1, -1)))

chi2fac00 = (kernel2(t2d*r2d*chimax)*D_chi(r2d*t2d*chimax))
chi2fac01 = (kernel2(1/t2d*r2d*chimax)*D_chi(r2d*1/t2d*chimax))
chi2fac01 = chi2fac01 * t2d**((4+nu_n_).reshape(1, 1, -1)-2)
chi2fac0  = chi2fac00 + chi2fac01


chifacs = w11*w12*chi1fac0* chi2fac0

result=np.zeros_like(ell_)
lmax = ell_.size
for ii  in range(ell_.size):        
    result[ii] = np.sum(chifacs*I4_ltrc[ii])

Clgg_c = chimax * result/np.pi**2/2.

print('Time taken = ', time.time()-begin)


In [ ]:

plt.loglog(ell_, Clgg_c)
plt.loglog(ell_,np.interp(ell_,cldd['ell'],cldd['dd'][0]))

In [ ]:
plt.plot(np.interp(ell_,cldd['ell'],cldd['dd'][0])/Clgg_c*4)
plt.xlim(0,100)
plt.ylim(0,1.1)

In [ ]:
print(1./(np.interp(ell_,cldd['ell'],cldd['dd'][0])/Clgg_c))

# Cross Correlations

computing $C_L^{\kappa g}$ in post Limber and compare to Limber results

In [ ]:
# set up (as before), but using FFTlog of P_deltapsi

r2d, t2d = np.meshgrid(t_,t_)
w11, w12 = np.meshgrid(w1,w1)
#nu axis
r2d, t2d = np.expand_dims(r2d, 2), np.expand_dims(t2d, 2)
w11, w12 = np.expand_dims(w11, 2), np.expand_dims(w12, 2)

# chimax and kernels
chimax   = chi_cmb

kernel1  = lensing_kernel
kernel2  = gal_kernel(Gauss_redshift(mean,width))

n = 2

begin=time.time()



chi1fac0 = (kernel1(r2d*chimax,chimax)*D_chi(r2d*chimax))
chi1fac0 = chi1fac0 *(r2d*chimax)**(1-(n+nu_n_.reshape(1, 1, -1)))

chi2fac00 = (kernel2(t2d*r2d*chimax)*D_chi(r2d*t2d*chimax))
chi2fac01 = (kernel2(1./t2d*r2d*chimax)*D_chi(r2d/t2d*chimax))
chi2fac01 = chi2fac01 * t2d**((n+nu_n_).reshape(1, 1, -1)-2)
chi2fac0  = chi2fac00 + chi2fac01

chifacs = w11*w12*chi1fac0* chi2fac0

result=np.zeros_like(ell_)
lmax = ell_.size
for ii  in range(ell_.size):        
    result[ii] = np.sum(chifacs*I2_ltrc[ii])
    

Cl_kg = chimax * result*1./np.pi**2/2.*prefac

print('Time taken = ', time.time()-begin)


In [ ]:
plt.loglog(ell_[:lmax],ell_[:lmax]*(ell_[:lmax]+1.)*Cl_kg[:lmax])
#plt.loglog(ll,clpp,ls='--', label='class old*4')
plt.loglog

In [ ]:
# cross in Limber
chis = np.linspace(1.,chi_cmb,5000) #in Mpc/h
ells = np.linspace(1,2000,200)
result=[]
for l_ in ells:
    integrand=[]
    for chi in chis:
        k = l_/chi #in h/Mpc
        try:
            Pk= cosmo.pk(k*h,0.)*D_chi(chi)**2*h**3
        except:
            Pk= 0.
        integrand+=[lensing_kernel(chi,chi_cmb)*kernel2(chi)*Pk]
    result+=[np.trapz(integrand, chis)]
clkg=np.asarray(result)*prefac

In [ ]:
#same factor of four as before, disagreement at low ell is expected
plt.loglog(ells,clkg*(ells+1.)/ells,label='Limber')
plt.loglog(ell_[:lmax],ell_[:lmax]*(ell_[:lmax]+1.)*Cl_kg[:lmax]/4.,label='no Limber/4')
plt.legend()

In [ ]:
# set up (as before), but using FFTlog of P_deltadelta, i.e. lensing kernel 2, should give better match to no Limber, also we should recover factor 0f 4

r2d, t2d = np.meshgrid(t_,t_)
w11, w12 = np.meshgrid(w1,w1)
#nu axis
r2d, t2d = np.expand_dims(r2d, 2), np.expand_dims(t2d, 2)
w11, w12 = np.expand_dims(w11, 2), np.expand_dims(w12, 2)

# chimax and kernels
chimax   = chi_cmb

kernel1  = lensing_kernel_2
kernel2  = gal_kernel(Gauss_redshift(mean,width))

n = 4

begin=time.time()



chi1fac0 = (kernel1(r2d*chimax,chimax)*D_chi(r2d*chimax))
chi1fac0 = chi1fac0 *(r2d*chimax)**(1-(n+nu_n_.reshape(1, 1, -1)))

chi2fac00 = (kernel2(t2d*r2d*chimax)*D_chi(r2d*t2d*chimax))
chi2fac01 = (kernel2(1./t2d*r2d*chimax)*D_chi(r2d/t2d*chimax))
chi2fac01 = chi2fac01 * t2d**((n+nu_n_).reshape(1, 1, -1)-2)
chi2fac0  = chi2fac00 + chi2fac01

chifacs = w11*w12*chi1fac0* chi2fac0

result=np.zeros_like(ell_)
lmax = ell_.size
for ii  in range(ell_.size):        
    result[ii] = np.sum(chifacs*I4_ltrc[ii])
    

Cl_kg2 = chimax * result*1./np.pi**2/2.*prefac

print('Time taken = ', time.time()-begin)

In [ ]:
plt.loglog(ells,clkg,label='Limber')
plt.loglog(ell_[:lmax],Cl_kg2[:lmax]/4.,label='no Limber')

In [ ]:
plt.plot(ell_[:lmax],np.interp(ell_[:lmax],ells,clkg)/Cl_kg2[:lmax]*4,label='ratio times 4')
plt.legend()